# Context retrieval 

In [1]:
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
import logging
import torch
from datetime import datetime
import transformers
import csv
# Configure logger
logger = logging.getLogger("contextgen")
logging.basicConfig(level=logging.INFO, format='%(name)s - %(levelname)s - %(message)s')
file_handler = logging.FileHandler("contextgen.log")
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(logging.Formatter('%(name)s - %(levelname)s - %(message)s'))
logger.addHandler(file_handler)


def embed_documents(docs, model_name="dunzhang/stella_en_1.5B_v5", device="cuda", verbose=False):
    """
    Embeds the input documents using the specified SentenceTransformer model.
    
    Args:
        docs (list of str): The documents to embed.
        model_name (str): The model name for SentenceTransformer.
        device (str): Device to load the model (e.g., "cuda" or "cpu").
        verbose (bool): Whether to print detailed logs.
    
    Returns:
        np.ndarray: Embeddings for the documents.
    """
    if verbose:
        logger.info(f"Loading model: {model_name} on {device}...")
    
    model = SentenceTransformer(model_name, trust_remote_code=True).to(device)
    
    if verbose:
        logger.info(f"Model loaded. Encoding {len(docs)} documents...")
    
    embeddings = model.encode(docs, convert_to_tensor=True).cpu().numpy()
    
    if verbose:
        logger.info("Encoding completed.")
    
    return embeddings


def cluster_documents_gpu(embeddings, docs,min_clusters=16, max_clusters=260, verbose=False):
    """
    Clusters the document embeddings using KMeans on GPU and selects the optimal
    number of clusters based on Silhouette Score.
    
    Args:
        embeddings (np.ndarray): Document embeddings.
        docs (list of str): Original documents for reference.
        max_clusters (int): Maximum number of clusters to evaluate.
        verbose (bool): Whether to log progress and details.
    
    Returns:
        dict: A dictionary with cluster IDs as keys and grouped documents as values.
    """
    if max_clusters is None:
        max_clusters = len(docs) - 1

    # Convert embeddings to PyTorch tensor on GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    embeddings_tensor = torch.tensor(embeddings, device=device)

    # Initial configurations
    best_silhouette_score = -1
    optimal_clusters = None
    clustered_docs = None

    if verbose:
        logger.info(f"Evaluating clustering for up to {max_clusters} clusters...")

    # Add progress bar for the clustering process
    for n_clusters in tqdm(range(min_clusters, max_clusters + 1), desc="Evaluating Clusters"):
        # Initialize cluster centers randomly
        centers = embeddings_tensor[torch.randperm(embeddings_tensor.size(0))[:n_clusters]]
        
        for iteration in range(10):  # Set max iterations
            # Compute distances and assign clusters
            distances = torch.cdist(embeddings_tensor, centers)
            cluster_assignments = torch.argmin(distances, dim=1)
            
            # Update cluster centers
            for i in range(n_clusters):
                cluster_points = embeddings_tensor[cluster_assignments == i]
                if len(cluster_points) > 0:
                    centers[i] = cluster_points.mean(dim=0)
        
        # Compute silhouette score on GPU
        cluster_assignments_np = cluster_assignments.cpu().numpy()
        try:
            silhouette_avg = silhouette_score(embeddings, cluster_assignments_np)
        except ValueError:
            silhouette_avg = -1  # Handle cases where silhouette score fails

        if verbose:
            logger.info(f"Clusters: {n_clusters}, Silhouette Score: {silhouette_avg:.4f}")

        # Check for the best silhouette score
        if silhouette_avg > best_silhouette_score:
            best_silhouette_score = silhouette_avg
            optimal_clusters = n_clusters
            best_cluster_assignments = cluster_assignments_np

    if optimal_clusters is not None:
        if verbose:
            logger.info(f"Optimal number of clusters determined: {optimal_clusters}")
        clustered_docs = {i: [] for i in range(optimal_clusters)}
        for idx, cluster in enumerate(best_cluster_assignments):
            clustered_docs[cluster].append(docs[idx])
    else:
        if verbose:
            logger.error("Failed to determine optimal clusters.")
    logger.info(f"Best Clusters: {optimal_clusters}, Best Silhouette Score: {best_silhouette_score:.4f}")

    return clustered_docs

# Model pipeline setup
MODEL_ID = "./Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=MODEL_ID,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
)

# Generate content
def generate_summary(contexts: list[str], target_tokens: int = 1000) -> str:
    """
    Summarize the given contexts to a target token size.

    Args:
        contexts (list[str]): List of context strings to summarize.
        target_tokens (int): Desired token size for the summarized output.

    Returns:
        str: Summarized text.
    """
    combined_context = " ".join(contexts)
    messages = [
        {"role": "system", "content": f"Summarize content clearly and concisely within {target_tokens} tokens, retaining key information."},
        {"role": "user", "content": f"Summarize the following text in under {target_tokens} tokens, focusing only on the essential points. \n\ntext:\n{combined_context}\n\nsummary:"},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=target_tokens,
    )

    return outputs[0]["generated_text"][-1]['content']

# Generate contextual retrieval summary
def generate_contextual_summary(document: str, chunk: str, target_tokens: int = 75) -> str:
    """
    Generate a succinct context to situate a chunk within the overall document for retrieval.

    Args:
        document (str): The full document as a string.
        chunk (str): The chunk of text for which context is being generated.
        target_tokens (int): Maximum token size for the output.

    Returns:
        str: Succinct context for the chunk.
    """
    messages = [
        {
            "role": "system",
            "content": f"Provide a succinct context to situate the following chunk within the overall document for retrieval. Limit the response to {target_tokens} tokens."
        },
        {
            "role": "user",
            "content": f"<document>\n{document}\n</document>\n\nHere is the chunk we want to situate within the whole document:\n<chunk>\n{chunk}\n</chunk>\n\nSuccinct result:"
        },
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=target_tokens,
    )

    return outputs[0]["generated_text"][-1]['content']
    
def run(docs):
    embeddings = embed_documents(docs, verbose=True)

    # Cluster the documents
    clustered_docs = cluster_documents_gpu(embeddings, docs,min_clusters=16, max_clusters=260,verbose=False)

    groups = [len(clustered_docs[i]) for i in range(len(clustered_docs))]
    # Calculate total sum
    total_sum = sum(groups)
    
    # Calculate proportions
    proportions = [round((x / total_sum) * 100, 2)/100 for x in groups]
    
    sum_data = []
    for idx, i in enumerate(proportions):
        length = int(32000*i)
        if length < 100:
            sum_data.append(' '.join(clustered_docs[idx]))
            logger.info(f"{groups[idx]} clusters : {len(sum_data[-1])} tokens")
        else:
            sum_data.append(generate_summary(clustered_docs[idx],length))
            logger.info(f"{groups[idx]} clusters : {length} tokens")
            
    # return sum_data
    file_name = "output.txt"

    # Open the file in write mode
    with open(file_name, "w", encoding="utf-8") as file:
        # Loop through each row and write it to the file
        file.write("|".join(sum_data))
    
    logger.info(f"Data successfully saved to {file_name}.")
    with open('output.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write the header row
        writer.writerow(['chunk', 'result'])
        
        # Process each chunk and write the results
        for i in range(len(docs)):
            ternimal = ''
            while ternimal != '.':
                chunk = docs[i]
                result = generate_contextual_summary(' '.join(sum_data), chunk)
                ternimal = result[-1]
                if ternimal == '.':
                    writer.writerow([chunk, result])
                    break
    

2024-12-08 22:31:47.988738: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 22:31:48.006866: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 22:31:48.026970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 22:31:48.032818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 22:31:48.050937: I tensorflow/core/platform/cpu_feature_guar

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

ds = load_dataset("MedRAG/textbooks")
df = ds['train'].to_pandas()  # Assuming you want the 'train' split
df.head()  # Display the first few rows

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

,id,title,content,contents
0,Anatomy_Gray_0,Anatomy_Gray,What is anatomy? Anatomy includes those struct...,Anatomy_Gray. What is anatomy? Anatomy include...
1,Anatomy_Gray_1,Anatomy_Gray,Observation and visualization are the primary ...,Anatomy_Gray. Observation and visualization ar...
2,Anatomy_Gray_2,Anatomy_Gray,How can gross anatomy be studied? The term ana...,Anatomy_Gray. How can gross anatomy be studied...
3,Anatomy_Gray_3,Anatomy_Gray,"This includes the vasculature, the nerves, the...","Anatomy_Gray. This includes the vasculature, t..."
4,Anatomy_Gray_4,Anatomy_Gray,Each of these approaches has benefits and defi...,Anatomy_Gray. Each of these approaches has ben...


In [3]:
anatomy_chunks = df[df['title']=='Anatomy_Gray']['content'].sort_index().to_list()

In [4]:
summary_data = run(anatomy_chunks)

contextgen - INFO - Loading model: dunzhang/stella_en_1.5B_v5 on cuda...
sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cuda
sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: dunzhang/stella_en_1.5B_v5
sentence_transformers.SentenceTransformer - INFO - 2 prompts are loaded, with the keys: ['s2p_query', 's2s_query']
contextgen - INFO - Model loaded. Encoding 3017 documents...


Batches:   0%|          | 0/95 [00:00<?, ?it/s]

contextgen - INFO - Encoding completed.
Evaluating Clusters: 100%|████████████████████| 245/245 [01:02<00:00,  3.90it/s]
contextgen - INFO - Best Clusters: 37, Best Silhouette Score: 0.0606
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 90 clusters : 953 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 42 clusters : 444 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 74 clusters : 784 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 66 clusters : 700 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 35 clusters : 371 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 11 clusters : 115 tokens
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
contextgen - INFO - 59 clusters : 627 token

In [6]:
# file_name = "output.txt"

# # Open the file in write mode
# with open(file_name, "w", encoding="utf-8") as file:
#     # Loop through each row and write it to the file
#     file.write("|".join(summary_data))

# print(f"Data successfully saved to {file_name}.")

Data successfully saved to output.txt.


In [72]:
import transformers
import torch

# Model pipeline setup
MODEL_ID = "./Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=MODEL_ID,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="cuda",
    pad_token_id=50256,
)

# Generate contextual retrieval summary
def generate_contextual_summary(document: str, chunk: str, target_tokens: int = 75) -> str:
    """
    Generate a succinct context to situate a chunk within the overall document for retrieval.

    Args:
        document (str): The full document as a string.
        chunk (str): The chunk of text for which context is being generated.
        target_tokens (int): Maximum token size for the output.

    Returns:
        str: Succinct context for the chunk.
    """
    messages = [
        {
            "role": "system",
            "content": f"Provide a succinct context to situate the following chunk within the overall document for retrieval. Limit the response to {target_tokens} tokens."
        },
        {
            "role": "user",
            "content": f"<document>\n{document}\n</document>\n\nHere is the chunk we want to situate within the whole document:\n<chunk>\n{chunk}\n</chunk>\n\nSuccinct result:"
        },
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=100,
    )

    return outputs[0]["generated_text"][-1]['content']


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
import csv


# Open the CSV file for writing
with open('output.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['chunk', 'result'])
    
    # Process each chunk and write the results
    for i in range(len(anatomy_chunks)):
        chunk = anatomy_chunks[i]
        result = generate_contextual_summary(' '.join(summary_data), chunk)
        writer.writerow([chunk, result])

print("Results have been written to output.csv")


Results have been written to output.csv


In [12]:
# Specify the file path
file_path = 'output.txt'

# Open the file in read mode and read its content
with open(file_path, 'r', encoding='utf-8') as file:
    content = file.read()



In [13]:
summary_data = content.replace('|','\n')

In [6]:
import pandas as pd
df = pd.read_csv('contextual_output.csv')
df.result.head()

0    The chunk to be situated within the whole docu...
1    To situate the provided chunk within the overa...
2    The provided chunk can be situated within the ...
3    The provided chunk is a summary of the approac...
4    The chunk can be situated within the document ...
Name: result, dtype: object

In [3]:
df1 = df.iloc[:,1:]
df1['contextual_chunk'] = df1['result']+df1['chunk']
df1

,chunk,result,contextual_chunk
0,What is anatomy? Anatomy includes those struct...,The chunk to be situated within the whole docu...,The chunk to be situated within the whole docu...
1,Observation and visualization are the primary ...,To situate the provided chunk within the overa...,To situate the provided chunk within the overa...
2,How can gross anatomy be studied? The term ana...,The provided chunk can be situated within the ...,The provided chunk can be situated within the ...
3,"This includes the vasculature, the nerves, the...",The provided chunk is a summary of the approac...,The provided chunk is a summary of the approac...
4,Each of these approaches has benefits and defi...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
...,...,...,...
3012,As the hippocampus resembles the appearance of...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3013,A second afferent pathway from the entorhinal ...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3014,Efferent axons leaving the hippocampal formati...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3015,Through this section we have described a colle...,The provided chunk is a continuation of the di...,The provided chunk is a continuation of the di...


In [4]:
df1.to_csv('contextual_output.csv',index=False)

In [8]:
s = df.result.apply(lambda x : x[-1] == '.').sum()
print(s)

3017


In [21]:
# Ensure you're using `loc` for assignments
t = 1
for idx, text in enumerate(df['result'].to_list()):
    if text[-1] != '.':  # Check if the last character is not a period
        print(f'finnish {t}/{s} ==== {round((t/s)*100,2)}%')
        terminal = text[-1]
        while terminal != '.':
            # Generate a contextual summary
            temp = generate_contextual_summary(summary_data, df['chunk'].iloc[idx])
            terminal = temp[-1]  # Get the last character of the generated text
            # print(terminal, idx)
            if terminal == '.':
                # Update the result column using loc
                df.loc[idx, 'result'] = temp
        t+=1
    if t % 100 == 0:
        df.to_csv(f'out{t}.csv',index=False)
df.to_csv('outputv2.csv',index=False)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


finnish 1/1701 ==== 0.06%


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


finnish 2/1701 ==== 0.12%


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

finnish 1314/1701 ==== 77.25%


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


finnish 1315/1701 ==== 77.31%


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


finnish 1316/1701 ==== 77.37%


In [16]:
df.to_csv('outputv3.csv',index=False)

In [17]:
import pandas as pd
df2 = pd.read_csv('outputv3.csv')
df2.head()

,Unnamed: 0,chunk,result
0,0,What is anatomy? Anatomy includes those struct...,The chunk to be situated within the whole docu...
1,1,Observation and visualization are the primary ...,To situate the provided chunk within the overa...
2,2,How can gross anatomy be studied? The term ana...,The provided chunk can be situated within the ...
3,3,"This includes the vasculature, the nerves, the...",The provided chunk is a summary of the approac...
4,4,Each of these approaches has benefits and defi...,The chunk can be situated within the document ...


In [18]:
df2.result.apply(lambda x : x[-1] == '.').sum()

1701

In [2]:
import pandas as pd
df2 = pd.read_csv('./results.csv')
df2.head()

,question,context,steps,final_ans
0,What is typically meant by the term anatomy wh...,What is anatomy? Anatomy includes those struct...,"['When the term anatomy is used by itself, it ...",Gross or macroscopic anatomy
1,What is microscopic anatomy also called?,What is anatomy? Anatomy includes those struct...,['##begin_quote## Anatomy includes those struc...,histology
2,What is the basis for the practice of medicine?,What is anatomy? Anatomy includes those struct...,"['To answer the question, we need to identify ...",Anatomy
3,What are the two primary techniques a student ...,Observation and visualization are the primary ...,['##begin_quote##Observation and visualization...,Observation and visualization
4,What is the difference between memorizing the ...,Observation and visualization are the primary ...,"['To study gross anatomy, one needs to underst...","To study gross anatomy, one needs to understan..."


In [13]:
df2.iloc[10,:]

question     What are the systems studied in a systemic app...
context      This includes the vasculature, the nerves, the...
steps        ['The question is asking for the systems studi...
final_ans    The systems studied in a systemic approach are...
Name: 10, dtype: object

In [14]:
df2.iloc[10,:].to_dict()

{'question': 'What are the systems studied in a systemic approach?',
 'context': 'This includes the vasculature, the nerves, the bones, the muscles, and all other structures and organs located in the region of the body defined as the thorax. After studying this region, the other regions of the body (i.e., the abdomen, pelvis, lower limb, upper limb, back, head, and neck) are studied in a similar fashion. In contrast, in a systemic approach, each system of the body is studied and followed throughout the entire body. For example, a study of the cardiovascular system looks at the heart and all of the blood vessels in the body. When this is completed, the nervous system (brain, spinal cord, and all the nerves) might be examined in detail. This approach continues for the whole body until every system, including the nervous, skeletal, muscular, gastrointestinal, respiratory, lymphatic, and reproductive systems, has been studied.',
 'steps': "['The question is asking for the systems studied i

In [17]:
df.iloc[100:102,1].to_list()

['Fig. 1.36 Dermatomes. C6 segment of spinal cordSpinal ganglionDermatomyotomeAutonomous region(where overlap ofdermatomes isleast likely)of C6 dermatome(pad of thumb)Skin on the lateral side of the forearm and on thethumb is innervated by C6 spinal level (spinal nerve).The dermis of the skin in this region develops from the somiteinitially associated with the C6 level of the developing spinal cordCaudalCranialSomite Fig. 1.37 Myotomes. C6 segment of spinal cordMuscles that abduct the arm are innervated by C5 and C6 spinal levels (spinal nerves) and develop from somites initially associated with C5 and C6 regions of developing spinal cordC5 segment of spinal cordDermatomyotomeSomite Fig. 1.38 Dermatomes. A. Anterior view. B. Posterior view. Fig. 1.39 Development of the visceral part of the nervous system.',
 'Fig. 1.38 Dermatomes. A. Anterior view. B. Posterior view. Fig. 1.39 Development of the visceral part of the nervous system. Motor nerve endingassociated withblood vessels,sweat g

In [3]:
sample = df2.head()

In [6]:
sam = df.iloc[:2,:]
sam['text'] = sam['result'] + sam['chunk'] 
sam

/tmp/ipykernel_3518804/3238176793.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sam['text'] = sam['result'] + sam['chunk']


,Unnamed: 0,chunk,result,text
0,0,What is anatomy? Anatomy includes those struct...,The chunk to be situated within the whole docu...,The chunk to be situated within the whole docu...
1,1,Observation and visualization are the primary ...,To situate the provided chunk within the overa...,To situate the provided chunk within the overa...


In [8]:
sample['Oracle Document'] = [sam['text'][0],sam['text'][0],sam['text'][0],sam['text'][1],sam['text'][1]]

/tmp/ipykernel_3518804/2957664865.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Oracle Document'] = [sam['text'][0],sam['text'][0],sam['text'][0],sam['text'][1],sam['text'][1]]


In [9]:
sample

,question,context,steps,final_ans,Oracle Document
0,What is typically meant by the term anatomy wh...,What is anatomy? Anatomy includes those struct...,"['When the term anatomy is used by itself, it ...",Gross or macroscopic anatomy,The chunk to be situated within the whole docu...
1,What is microscopic anatomy also called?,What is anatomy? Anatomy includes those struct...,['##begin_quote## Anatomy includes those struc...,histology,The chunk to be situated within the whole docu...
2,What is the basis for the practice of medicine?,What is anatomy? Anatomy includes those struct...,"['To answer the question, we need to identify ...",Anatomy,The chunk to be situated within the whole docu...
3,What are the two primary techniques a student ...,Observation and visualization are the primary ...,['##begin_quote##Observation and visualization...,Observation and visualization,To situate the provided chunk within the overa...
4,What is the difference between memorizing the ...,Observation and visualization are the primary ...,"['To study gross anatomy, one needs to underst...","To study gross anatomy, one needs to understan...",To situate the provided chunk within the overa...


In [10]:
sample['Distractor Documents'] = [df.chunk[200:202],df.chunk[203:205],df.chunk[206:207],df.chunk[208:209],df.chunk[210:212]]

/tmp/ipykernel_3518804/2695756270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['Distractor Documents'] = [df.chunk[200:202],df.chunk[203:205],df.chunk[206:207],df.chunk[208:209],df.chunk[210:212]]


In [11]:
sample

,question,context,steps,final_ans,Oracle Document,Distractor Documents
0,What is typically meant by the term anatomy wh...,What is anatomy? Anatomy includes those struct...,"['When the term anatomy is used by itself, it ...",Gross or macroscopic anatomy,The chunk to be situated within the whole docu...,"200 Each spinal nerve divides, as it emerge..."
1,What is microscopic anatomy also called?,What is anatomy? Anatomy includes those struct...,['##begin_quote## Anatomy includes those struc...,histology,The chunk to be situated within the whole docu...,203 Surface features of the back are used t...
2,What is the basis for the practice of medicine?,What is anatomy? Anatomy includes those struct...,"['To answer the question, we need to identify ...",Anatomy,The chunk to be situated within the whole docu...,206 Extending between CVII and the external...
3,What are the two primary techniques a student ...,Observation and visualization are the primary ...,['##begin_quote##Observation and visualization...,Observation and visualization,To situate the provided chunk within the overa...,208 The tip of the coccyx is palpable at th...
4,What is the difference between memorizing the ...,Observation and visualization are the primary ...,"['To study gross anatomy, one needs to underst...","To study gross anatomy, one needs to understan...",To situate the provided chunk within the overa...,210 Because the subarachnoid space can be a...


In [21]:
sample = sample.rename(columns={"final_ans": "Final Answer", "steps": "COT.", "question": "Question"})

In [22]:
new_index = [9047,9048,9049,9050,9051]
sample.index = new_index

In [24]:
sample.iloc[:,[0,2,3,4,5]]

,Question,COT.,Final Answer,Oracle Document,Distractor Documents
9047,What is typically meant by the term anatomy wh...,"['When the term anatomy is used by itself, it ...",Gross or macroscopic anatomy,The chunk to be situated within the whole docu...,"200 Each spinal nerve divides, as it emerge..."
9048,What is microscopic anatomy also called?,['##begin_quote## Anatomy includes those struc...,histology,The chunk to be situated within the whole docu...,203 Surface features of the back are used t...
9049,What is the basis for the practice of medicine?,"['To answer the question, we need to identify ...",Anatomy,The chunk to be situated within the whole docu...,206 Extending between CVII and the external...
9050,What are the two primary techniques a student ...,['##begin_quote##Observation and visualization...,Observation and visualization,To situate the provided chunk within the overa...,208 The tip of the coccyx is palpable at th...
9051,What is the difference between memorizing the ...,"['To study gross anatomy, one needs to underst...","To study gross anatomy, one needs to understan...",To situate the provided chunk within the overa...,210 Because the subarachnoid space can be a...


In [13]:
import pandas as pd
df = pd.read_csv('craft.csv')
df.tail()

,question,context,steps,final_ans
9037,What is the order of the structures in the Pap...,The provided chunk is a continuation of the di...,"['First, identify the starting point of the Pa...",The order of the structures in the Papez circu...
9038,What is the name of the tract that connects th...,The provided chunk is a continuation of the di...,"['First, we need to identify the part of the P...",mammillothalamic tract
9039,What is the main pathway described by Papez's ...,The chunk can be situated within the whole doc...,"[""The question asks for the main pathway descr...",The main pathway described by Papez's circuit ...
9040,What is the function of the cingulum fibers in...,The chunk can be situated within the whole doc...,['##begin_quote##The cingulum fibers from the ...,The cingulum fibers in the Papez circuit are i...
9041,What role do some of the structures in the Pap...,The chunk can be situated within the whole doc...,['The question asks about the role of some str...,The Papez pathway has a mixed role in the expr...


In [12]:
import pandas as pd
df1 = pd.read_csv('contextual_output.csv')
df1.tail()

,chunk,result,contextual_chunk
3012,As the hippocampus resembles the appearance of...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3013,A second afferent pathway from the entorhinal ...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3014,Efferent axons leaving the hippocampal formati...,The chunk can be situated within the document ...,The chunk can be situated within the document ...
3015,Through this section we have described a colle...,The provided chunk is a continuation of the di...,The provided chunk is a continuation of the di...
3016,the internal capsule to the cingulate gyrus. L...,The chunk can be situated within the whole doc...,The chunk can be situated within the whole doc...


In [14]:
for idx , i in enumerate(list(df.isnull().sum(axis=1))):
    if i != 0:
        print(idx)
    

2436
2779
4772
4842
5059
5282


In [17]:
df.iloc[2779].context

'The provided chunk can be situated within the document in the section discussing the urinary system, specifically in the subsection on "Urological Procedures". The chunk would fit within the context of explaining the nephrostomy procedure and its application in the clinic.In the clinic A nephrostomy is a procedure where a tube is placed through the lateral or posterior abdominal wall into the renal cortex to lie within the renal pelvis. The function of this tube is to allow drainage of urine from the renal pelvis through the tube externally (Fig. 4.160). The kidneys are situated on the posterior abdominal wall, and in thin healthy subjects may be only up to 2 to 3 cm from the skin. Access to the kidney is relatively straightforward, because the kidney can be easily visualized under ultrasound guidance. Using local anesthetic, a needle can be placed, under ultrasound direction, through the skin into the renal cortex and into the renal pelvis. A series of wires and tubes can be passed t

In [68]:
import pandas as pd 
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
import logging
import torch
from datetime import datetime
import transformers
import csv

In [141]:
data = pd.read_csv('craft_finalv2.csv')
data['neg_context'] = ''
data['full_context'] = ''
docs = list(data.context.unique())

In [142]:
data.head()

,question,context,steps,final_ans,neg_context,full_context
0,What is the primary focus of gross anatomy in ...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",The primary focus of gross anatomy in the fiel...,,
1,What is the term for the study of cells and ti...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",histology,,
2,What is the role of anatomy in the practice of...,The chunk to be situated within the whole docu...,['##begin_quote##The term anatomy tends to mea...,Anatomy forms the basis for the practice of me...,,
3,What are the two primary techniques a student ...,To situate the provided chunk within the overa...,"['To answer the question, we need to identify ...",observation and visualization,,
4,How is anatomy more than just memorization of ...,To situate the provided chunk within the overa...,['##begin_quote## Anatomy is much more than ju...,Gross anatomy should be studied using observat...,,


In [108]:
model = SentenceTransformer("dunzhang/stella_en_1.5B_v5", trust_remote_code=True).cuda()
doc_embeddings = model.encode(docs,show_progress_bar=True)
model.similarity_fn_name = "euclidean"

Batches:   0%|          | 0/95 [00:00<?, ?it/s]

In [143]:
import random
mem_doc = doc_embeddings.shape[0]
context_neg = {}
for i in tqdm(range(mem_doc), desc="similarity calculate "):
    query = doc_embeddings[i]
    similarities = torch.abs(model.similarity(query, doc_embeddings))
    top_values, top_indices = torch.topk(similarities, k=3)
    neg_text = ["["+docs[j]+"]" for j in list(top_indices[0])]
    full_context = []
    temp = "[" + docs[i] + "]"
    full_context.extend(neg_text)
    full_context.append(temp)
    random.shuffle(full_context)
    context_neg[docs[i]] = neg_text
    idx_list = data[data.context == docs[i]].index.to_list()
    data.iloc[idx_list,-2] = ','.join(neg_text)
    data.iloc[idx_list,-1] = ','.join(full_context)
print('finish')

similarity calculate : 100%|███████████████| 3017/3017 [00:04<00:00, 635.82it/s]

finish


In [110]:
data

,question,context,steps,final_ans,neg_context
0,What is the primary focus of gross anatomy in ...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",The primary focus of gross anatomy in the fiel...,['The chunk is a clinical note from a surgical...
1,What is the term for the study of cells and ti...,The chunk to be situated within the whole docu...,"['To answer the question, we need to understan...",histology,['The chunk is a clinical note from a surgical...
2,What is the role of anatomy in the practice of...,The chunk to be situated within the whole docu...,['##begin_quote##The term anatomy tends to mea...,Anatomy forms the basis for the practice of me...,['The chunk is a clinical note from a surgical...
3,What are the two primary techniques a student ...,To situate the provided chunk within the overa...,"['To answer the question, we need to identify ...",observation and visualization,"['The chunk describes the sartorius muscle, it..."
4,How is anatomy more than just memorization of ...,To situate the provided chunk within the overa...,['##begin_quote## Anatomy is much more than ju...,Gross anatomy should be studied using observat...,"['The chunk describes the sartorius muscle, it..."
...,...,...,...,...,...
9049,What is the order of the structures in the Pap...,The provided chunk is a continuation of the di...,"['First, identify the starting point of the Pa...",The order of the structures in the Papez circu...,['The chunk is a clinical note from a surgical...
9050,What is the name of the tract that connects th...,The provided chunk is a continuation of the di...,"['First, we need to identify the part of the P...",mammillothalamic tract,['The chunk is a clinical note from a surgical...
9051,What is the main pathway described by Papez's ...,The chunk can be situated within the whole doc...,"[""The question asks for the main pathway descr...",The main pathway described by Papez's circuit ...,['The chunk is a clinical note from a surgical...
9052,What is the function of the cingulum fibers in...,The chunk can be situated within the whole doc...,['##begin_quote##The cingulum fibers from the ...,The cingulum fibers in the Papez circuit are i...,['The chunk is a clinical note from a surgical...


In [113]:
data.to_csv('last_craft.csv', index=False)

In [144]:
data.tail()

,question,context,steps,final_ans,neg_context,full_context
9049,What is the order of the structures in the Pap...,The provided chunk is a continuation of the di...,"['First, identify the starting point of the Pa...",The order of the structures in the Papez circu...,[The chunk is a clinical note from a surgical ...,[The provided chunk is a description of how a ...
9050,What is the name of the tract that connects th...,The provided chunk is a continuation of the di...,"['First, we need to identify the part of the P...",mammillothalamic tract,[The chunk is a clinical note from a surgical ...,[The provided chunk is a description of how a ...
9051,What is the main pathway described by Papez's ...,The chunk can be situated within the whole doc...,"[""The question asks for the main pathway descr...",The main pathway described by Papez's circuit ...,[The chunk is a clinical note from a surgical ...,[The chunk can be situated within the whole do...
9052,What is the function of the cingulum fibers in...,The chunk can be situated within the whole doc...,['##begin_quote##The cingulum fibers from the ...,The cingulum fibers in the Papez circuit are i...,[The chunk is a clinical note from a surgical ...,[The chunk can be situated within the whole do...
9053,What role do some of the structures in the Pap...,The chunk can be situated within the whole doc...,['The question asks about the role of some str...,The Papez pathway has a mixed role in the expr...,[The chunk is a clinical note from a surgical ...,[The chunk can be situated within the whole do...


In [145]:
data.steps = data.steps.apply(lambda x : x.replace("', '",' ').replace("['",'').replace("']",''))

In [146]:
data.question = data.question.apply(lambda x : 'Question: '+x+'\n\n')
data.full_context = data.full_context.apply(lambda x : 'context: '+x+'\n\n')
data.steps = data.steps.apply(lambda x : '<REASON>'+x+'</REASON>')
data.final_ans = data.final_ans.apply(lambda x : '<ANSWER>'+x+'</ANSWER>')
data.tail(2)

,question,context,steps,final_ans,neg_context,full_context
9052,Question: What is the function of the cingulum...,The chunk can be situated within the whole doc...,<REASON>##begin_quote##The cingulum fibers fro...,<ANSWER>The cingulum fibers in the Papez circu...,[The chunk is a clinical note from a surgical ...,context: [The chunk can be situated within the...
9053,Question: What role do some of the structures ...,The chunk can be situated within the whole doc...,<REASON>The question asks about the role of so...,<ANSWER>The Papez pathway has a mixed role in ...,[The chunk is a clinical note from a surgical ...,context: [The chunk can be situated within the...


In [155]:
data['instruction'] = data['question'] + data['full_context']
data['output'] = data['steps'] + data['final_ans']

In [157]:
data1 = data.loc[:,['instruction','output']]
# data.columns

In [173]:
data1['output'] = data1['output'].apply(lambda x: x.replace('[','').replace(']',''))

In [176]:
data1.to_csv('Dataset_craft.csv', index=False)

In [ ]:
"Answer user's question using the information given in given context. Here are things to pay attention to:"
"- First provide step-by-step reasoning on how to answer the question."
"- In the reasoning, if you need to copy paste some sentences from the context, include them in ##begin_quote## and ##end_quote##." 
"This would mean that things outside of ##begin_quote## and ##end_quote## are not directly copy paste from the context."
"- End your response with final answer in the form <ANSWER>: $answer, the answer should be succinct."

In [177]:
data1

,instruction,output
0,Question: What is the primary focus of gross a...,"<REASON>To answer the question, we need to und..."
1,Question: What is the term for the study of ce...,"<REASON>To answer the question, we need to und..."
2,Question: What is the role of anatomy in the p...,<REASON>##begin_quote##The term anatomy tends ...
3,Question: What are the two primary techniques ...,"<REASON>To answer the question, we need to ide..."
4,Question: How is anatomy more than just memori...,<REASON>##begin_quote## Anatomy is much more t...
...,...,...
9049,Question: What is the order of the structures ...,"<REASON>First, identify the starting point of ..."
9050,Question: What is the name of the tract that c...,"<REASON>First, we need to identify the part of..."
9051,Question: What is the main pathway described b...,"<REASON>""The question asks for the main pathwa..."
9052,Question: What is the function of the cingulum...,<REASON>##begin_quote##The cingulum fibers fro...
